In [1]:
import sqlite3
import pandas as pd
import re
import os

In [2]:
# read table with metadata from file through pandas

metadata = pd.read_csv(os.path.join('.', 'data', 'genotyping_data', 'metadata.csv'), index_col=0)
metadata

,dna_chip_id,breed,sex
0,202290551164R09C01,Д,Хр
1,202341831114R02C01,Д,Хр
2,202341831114R03C01,Д,Хр
3,202341831114R04C01,Д,Хр
4,202290551140R01C01,Д,Хр
...,...,...,...
836,203071331169R10C02,Д,Хр
837,203071331169R12C02,Д,Хр
838,202341831114R01C01,Д,Хр
839,202341831114R06C01,Д,Хр


In [77]:
# read basic table from file through pandas

genstudio = pd.read_csv(os.path.join('.', 'data', 'genotyping_data','genstudio.csv'), index_col=0, low_memory=False)
genstudio

,SNP Name,SNP Index,SNP Aux,Sample ID,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,Position,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio
0,1_10573221,1,0,202341831114R01C01,[T/C],-,-,-,-,-,-,1,10573221,0.0000,0.0000,0.942,0.413,1.0000,0.4040
1,1_10673082,2,0,202341831114R01C01,[T/C],A,A,T,T,A,A,1,10673082,0.8272,0.8076,0.039,0.968,0.0000,0.3017
2,1_10723065,3,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,10723065,0.8316,0.8107,0.011,1.577,0.0000,0.0388
3,1_11337555,4,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,11337555,0.3781,0.7925,0.045,1.104,0.0000,0.2761
4,1_11407894,5,0,202341831114R01C01,[A/G],G,G,G,G,B,B,1,11407894,0.9038,0.8670,0.983,1.122,0.9994,0.0022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,ALGA0111890,13032,0,202341831127R03C02,[A/G],A,G,T,C,A,B,6,90920481,0.8159,0.8002,0.581,1.572,0.5540,-0.0907
1999996,ALGA0111894,13033,0,202341831127R03C02,[A/G],A,G,T,C,A,B,13,73955342,0.9596,0.9298,0.468,1.415,0.4746,0.0558
1999997,ALGA0111899,13034,0,202341831127R03C02,[A/G],A,G,A,G,A,B,UWGS,118787518,0.8900,0.8548,0.502,1.282,0.5313,-0.0773
1999998,ALGA0111904,13035,0,202341831127R03C02,[A/G],G,G,C,C,B,B,6,17436645,0.8433,0.8187,0.941,1.180,1.0000,0.0258


In [78]:
# transform values from column 'Position' into two columns 'start' and 'end'

for i in range(genstudio.shape[0]):
    start_coordinate = re.compile(r'\d+\-').findall(genstudio.loc[i, 'Position'])
    if start_coordinate == []:
        genstudio.loc[i, 'Position_start'] = genstudio.loc[i, 'Position']
    else:
        genstudio.loc[i, 'Position_start'] = start_coordinate[0][:-1]
    
    end_coordinate = re.compile(r'\-\d+').findall(genstudio.loc[i, 'Position'])
    if end_coordinate == []:
        genstudio.loc[i, 'Position_end'] = genstudio.loc[i, 'Position']
    else:
        genstudio.loc[i, 'Position_end'] = end_coordinate[0][1:]

genstudio.drop('Position', axis=1, inplace=True)

genstudio.Position_start = genstudio.Position_start.astype(int)
genstudio.Position_end = genstudio.Position_end.astype(int)
genstudio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000000 entries, 0 to 1999999
Data columns (total 20 columns):
 #   Column             Dtype  
---  ------             -----  
 0   SNP Name           object 
 1   SNP Index          int64  
 2   SNP Aux            int64  
 3   Sample ID          object 
 4   SNP                object 
 5   Allele1 - Top      object 
 6   Allele2 - Top      object 
 7   Allele1 - Forward  object 
 8   Allele2 - Forward  object 
 9   Allele1 - AB       object 
 10  Allele2 - AB       object 
 11  Chr                object 
 12  GC Score           float64
 13  GT Score           float64
 14  Theta              float64
 15  R                  float64
 16  B Allele Freq      float64
 17  Log R Ratio        float64
 18  Position_start     int64  
 19  Position_end       int64  
dtypes: float64(6), int64(4), object(10)
memory usage: 384.9+ MB


In [84]:
# create sql database and save two tables there

connection = sqlite3.connect('genotyping_data.db')

genstudio.to_sql('genstudio', connection, if_exists='replace', index=False)
metadata.to_sql('metadata', connection, if_exists='replace', index=False)

841

In [85]:
# select from the database example

select_query = '''
                SELECT Chr, Position_start, Position_end
                FROM   genstudio
                '''

result = connection.execute(select_query).fetchall()
result[:5]

[('1', 10573221, 10573221),
 ('1', 10673082, 10673082),
 ('1', 10723065, 10723065),
 ('1', 11337555, 11337555),
 ('1', 11407894, 11407894)]

In [87]:
# select unique values

connection.execute('''SELECT DISTINCT Chr FROM genstudio''').fetchall()

[('1',),
 ('10',),
 ('11',),
 ('12',),
 ('13',),
 ('14',),
 ('15',),
 ('16',),
 ('17',),
 ('18',),
 ('0',),
 ('2',),
 ('3',),
 ('4',),
 ('5',),
 ('6',),
 ('7',),
 ('8',),
 ('9',),
 ('UWGS',),
 ('X',),
 ('Y',)]

In [102]:
# select limited number from the top

result2 = connection.execute('''SELECT *
                                FROM genstudio
                                WHERE `Log R Ratio` > 0.5
                                LIMIT 3''').fetchall()

pd.DataFrame(result2, columns=genstudio.columns)

,SNP Name,SNP Index,SNP Aux,Sample ID,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio,Position_start,Position_end
0,1_2812415,18,0,202341831114R01C01,[T/C],A,A,T,T,A,A,1,0.4271,0.5849,0.020,1.713,0.0,0.5524,2812415,2812415
1,1_301198363,22,0,202341831114R01C01,[A/C],-,-,-,-,-,-,1,0.0000,0.0000,0.070,0.591,0.0,0.5216,301198363,301198363
2,1_303828731,24,0,202341831114R01C01,[A/G],G,G,G,G,B,B,1,0.5483,0.6523,0.948,1.349,1.0,0.5307,303828731,303828731


In [103]:
# select by pattern

connection.execute('''SELECT *
                      FROM genstudio
                      WHERE SNP LIKE "_A%"''').fetchall()[0]

('1_10723065',
 3,
 0,
 '202341831114R01C01',
 '[A/G]',
 'A',
 'A',
 'T',
 'T',
 'A',
 'A',
 '1',
 0.8316,
 0.8107,
 0.011,
 1.577,
 0.0,
 0.0388,
 10723065,
 10723065)

In [97]:
# insersion into the database example

metadata_new = [['unique_id', 'new_breed', 'another_sex'],
                ['one_more_unique_id', 'brand_new_breed', 'one_more_sex']]

insersion_query = '''INSERT INTO
                     metadata(dna_chip_id, breed, sex)
                     VALUES(?, ?, ?)'''

for dna_chip_id, breed, sex in metadata_new:
    connection.execute(insersion_query, (dna_chip_id, breed, sex))

connection.commit()

# or connection.executemany(insersion_query, metadata_new)

In [98]:
# combine two tables, based on the common column with sample_id/dna_chip_id

query = '''CREATE TABLE genstudio_with_metadata AS
                SELECT genstudio.*, metadata.breed, metadata.sex
                FROM   genstudio 
                INNER JOIN metadata 
                        ON genstudio.[Sample ID] = metadata.dna_chip_id'''

connection.execute(query)
connection.commit()

In [ ]:
connection.close()